In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import re

In [6]:

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# YOUR CODE HERE
nhl_df = nhl_df[nhl_df['year'] == 2018]
cities = pd.read_html('assets/wikipedia_data.html')[1]
#cities.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
new_nhl = cities[['Metropolitan area', 'NHL']]
new_nhl['LName'] = new_nhl['NHL'].apply(lambda x: re.findall('[A-Z][a-z]+', x))
new_nhl = new_nhl.explode(column='LName').drop('NHL', axis=1).dropna()
new_nhl.rename({'LName': 'team_LName'}, axis=1, inplace=True)
nhlteams_index = new_nhl['Metropolitan area'].unique()
nhl_df['team_LName'] = nhl_df['team'].apply(lambda x: re.findall('.+\s(\w+)\*?', x)[-1])
nhl_cities = nhl_df.merge(new_nhl, right_on='team_LName', left_on='team_LName')
nhl_cities['W'] = nhl_cities['W'].astype(int)
nhl_cities['L'] = nhl_cities['L'].astype(int)
nhl_cities['W/L_%'] = nhl_cities['W'] / (nhl_cities['L'] + nhl_cities['W'])
nhl_cities_merged = nhl_cities.groupby('Metropolitan area').agg({'W/L_%': 'mean'}).sort_index()
q1 = nhl_cities_merged.merge(cities[['Metropolitan area', 'Population (2016 est.)[8]']], on='Metropolitan area')
q1 = q1.set_index('Metropolitan area').loc[nhlteams_index]
q1['Population (2016 est.)[8]'] = q1['Population (2016 est.)[8]'].astype(np.float)

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [7]:
q1

,W/L_%,Population (2016 est.)[8]
Metropolitan area,,
New York City,0.518201,20153634.0
Los Angeles,0.622895,13310447.0
San Francisco Bay Area,0.625000,6657982.0
Chicago,0.458333,9512999.0
Dallas–Fort Worth,0.567568,7233323.0
"Washington, D.C.",0.653333,6131977.0
Philadelphia,0.617647,6070500.0
Boston,0.714286,4794447.0
Minneapolis–Saint Paul,0.633803,3551036.0


In [6]:
population_by_region = q1['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss_by_region = q1['W/L_%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012486162921209909

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
        
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities_df=pd.read_html("assets/wikipedia_data.html")[1]
    cities_df=cities_df.iloc[:-1,[0,3,5,6,7,8]]
    cities_df.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
    
    mlb_df = pd.read_csv('assets/mlb.csv')
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    new_mlb = cities_df[['Metropolitan area', 'MLB']]
    new_mlb.loc[:, 'team_LName'] = cities_df.loc[:, 'MLB'].apply(lambda x: re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x))
    new_mlb = new_mlb.explode(column='team_LName').dropna().drop('MLB', axis=1)
    mlb_index = new_mlb['Metropolitan area'].unique()
    mlb_df.loc[:, 'team_LName'] = mlb_df.loc[:, 'team'].apply(lambda x: re.findall('.+\s([A-Z][a-z]+)', x)[0])
    mlb_df.loc[0, 'team_LName'] = 'Red Sox'
    mlb_df.loc[8, 'team_LName'] = 'White Sox'
    mlb_df.loc[3, 'team_LName'] = 'Blue Jays'
    q3 = new_mlb.merge(mlb_df, on='team_LName').groupby('Metropolitan area').agg({'W-L%': 'mean'}).loc[mlb_index]
    q3 = q3.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')
    q3['W-L%'] = q3['W-L%'].astype(np.float)
    q3['Population'] = q3['Population'].astype(np.float)
    
    population_by_region = q3['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = q3['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

    raise NotImplementedError()
    
mlb_correlation()    

0.15003737475409495

In [12]:
def q5():
    import pandas as pd
    import numpy as np
    import re
    from scipy import stats
    
    cities_df = pd.read_html('assets/wikipedia_data.html')[1]
    cities_df.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
    cities_df = cities_df.iloc[:-1, :]
    cities_df['Population'] = cities_df['Population'].astype(np.float)
    
    
    nhl_df = pd.read_csv('assets/nhl.csv')
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    metro_nhl = cities_df[['Metropolitan area', 'NHL']]
    metro_nhl['team_LName'] = metro_nhl['NHL'].apply(lambda x: re.findall('[A-Z][a-z]+', x))
    metro_nhl = metro_nhl.explode(column='team_LName').dropna()
    metro_nhl.drop('NHL', axis=1, inplace=True)
    nhl_index = metro_nhl['Metropolitan area'].unique()
    nhl_df['team_LName'] = nhl_df['team'].apply(lambda x: re.findall('.+\s([A-Z1-9]+[a-z]+)\*?', x)[0])
    merged_nhl = metro_nhl.merge(nhl_df, on='team_LName')
    merged_nhl['W'] = merged_nhl['W'].astype(np.float)
    merged_nhl['L'] = merged_nhl['L'].astype(np.float)
    merged_nhl['W/L%'] = merged_nhl['W'] / (merged_nhl['W'] + merged_nhl['L'])
    grouped_nhl = merged_nhl.groupby('Metropolitan area').agg({'W/L%': 'mean'}).merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area')
    grouped_nhl.set_index('Metropolitan area', inplace=True)
    q1 = grouped_nhl.loc[nhl_index, :]
    
    
    nba_df = pd.read_csv('assets/nba.csv')
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['W/L%'] = nba_df['W/L%'].astype(np.float)
    metro_nba = cities_df[['Metropolitan area', 'NBA']]
    metro_nba['team_LName'] = metro_nba['NBA'].apply(lambda x: re.findall('[A-Z1-9]+[a-z]+', x))
    metro_nba = metro_nba.explode(column='team_LName').dropna()
    metro_nba.drop('NBA', axis=1, inplace=True)
    nba_index = metro_nba['Metropolitan area'].unique()
    nba_df['team_LName'] = nba_df['team'].apply(lambda x: re.findall('.*\s([A-Z1-9]+[a-z]+)\*?\s\S', x)[0])
    merged_nba = nba_df.merge(metro_nba, on='team_LName')
    grouped_nba = merged_nba.groupby(by='Metropolitan area').agg({'W/L%': 'mean'})
    q2 = grouped_nba.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')
    q2 = q2.loc[nba_index, :]
    
    
    mlb_df = pd.read_csv('assets/mlb.csv')
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    metro_mlb = cities_df[['Metropolitan area', 'MLB']]
    metro_mlb['team_LName'] = metro_mlb['MLB'].apply(lambda x: re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z1-9]+[a-z]+', x))
    metro_mlb = metro_mlb.explode(column='team_LName').dropna().drop('MLB', axis=1)
    mlb_index = metro_mlb['Metropolitan area'].unique()
    mlb_df['team_LName'] = mlb_df['team'].apply(lambda x: re.findall('.+\s([A-Z][a-z]+)', x)[0])
    mlb_df.iloc[0, -1] = 'Red Sox'
    mlb_df.iloc[8, -1] = 'White Sox'
    mlb_df.iloc[3, -1] = 'Blue Jays'
    merged_mlb = mlb_df.merge(metro_mlb, on='team_LName')
    grouped_mlb = merged_mlb.groupby(by='Metropolitan area').agg({'W-L%': 'mean'})
    q3 = grouped_mlb.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')
    q3 = q3.loc[mlb_index, :]
    
    
    nfl_df = pd.read_csv('assets/nfl.csv')
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df.drop([0, 5, 10, 15, 20, 25, 30, 35], axis=0, inplace=True)
    nfl_df['W-L%'] = nfl_df['W-L%'].astype(np.float)
    metro_nfl = cities_df[['Metropolitan area', 'NFL']]
    metro_nfl['team_LName'] = metro_nfl['NFL'].apply(lambda x: re.findall('[A-Z1-9]+[a-z]+', x))
    metro_nfl = metro_nfl.explode(column='team_LName').dropna().drop('NFL', axis=1)
    nfl_index = metro_nfl['Metropolitan area'].unique()
    nfl_df['team_LName'] = nfl_df['team'].apply(lambda x: re.findall('.*\s([A-Z1-9]+[a-z]+)\S*', x)[0])
    merged_nfl = nfl_df.merge(metro_nfl, on='team_LName')
    grouped_nfl = merged_nfl.groupby(by='Metropolitan area').agg({'W-L%': 'mean'})
    q4 = grouped_nfl.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')
    q4 = q4.loc[nfl_index, :]
    
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    q1_nhl = q1.copy()
    q2_nba = q2.copy()
    q3_mlb = q3.copy()
    q4_nfl = q4.copy()
    
    nfl_nba = pd.merge(q4_nfl, q2_nba, right_index=True, left_index=True)
    p_values.loc['NFL', 'NBA'] = stats.ttest_rel(nfl_nba.iloc[:, 0], nfl_nba.iloc[:, 2])[1]
    p_values.loc['NBA', 'NFL'] = p_values.loc['NFL', 'NBA']
    
    nfl_nhl = pd.merge(q1_nhl, q4_nfl, right_index=True, left_index=True)
    p_values.loc['NHL', 'NFL'] = stats.ttest_rel(nfl_nhl.iloc[:, 0], nfl_nhl.iloc[:, 2])[1]
    p_values.loc['NFL', 'NHL'] = p_values.loc['NHL', 'NFL']
    
    nfl_mlb = pd.merge(q3_mlb, q4_nfl, right_index=True, left_index=True)
    p_values.loc['NFL', 'MLB'] = stats.ttest_rel(nfl_mlb.iloc[:, 0], nfl_mlb.iloc[:, 2])[1]
    p_values.loc['MLB', 'NFL'] = p_values.loc['NFL', 'MLB']
    
    nba_nhl = pd.merge(q1_nhl, q2_nba, right_index=True, left_index=True)
    p_values.loc['NBA', 'NHL'] = stats.ttest_rel(nba_nhl.iloc[:, 0], nba_nhl.iloc[:, 2])[1]
    p_values.loc['NHL', 'NBA'] = p_values.loc['NBA', 'NHL']
    
    nba_mlb = pd.merge(q2_nba, q3_mlb, left_index=True, right_index=True)
    p_values.loc['NBA', 'MLB'] = stats.ttest_rel(nba_mlb.iloc[:, 0], nba_mlb.iloc[:, 2])[1]
    p_values.loc['MLB', 'NBA'] = p_values.loc['NBA', 'MLB']
    
    nhl_mlb = pd.merge(q1_nhl, q3_mlb, left_index=True, right_index=True)
    p_values.loc['NHL', 'MLB'] = stats.ttest_rel(nhl_mlb.iloc[:, 0], nhl_mlb.iloc[:, 2])[1]
    p_values.loc['MLB', 'NHL'] = p_values.loc['NHL', 'MLB']
    
    return p_values
    
    
q5()

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


In [13]:
p_values = q5()

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [14]:
1e-2

0.01

In [15]:
p_values.loc['NBA', 'NHL'] - 0.02 < 1e-2

True